<a href="https://colab.research.google.com/github/giuliapuntoit/CapsnetGoogleLandmark/blob/main/project_CapsNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install and import requirements**

In [ ]:
!pip install tensorboardX
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
from tqdm import tqdm
import os
from tensorboardX import SummaryWriter
from datetime import datetime

     |████████████████████████████████| 204kB 2.9MB/s 


**Download images**

In [ ]:
# Note to Kagglers: This script will not run directly in Kaggle kernels. You
# need to download it and run it on your local machine.

# Downloads images from the Google Landmarks dataset using multiple threads.
# Images that already exist will not be downloaded again, so the script can
# resume a partially completed download. All images will be saved in the JPG
# format with 90% compression quality.

import sys, os, multiprocessing, csv
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
import torch
from torchvision import transforms


def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_label_list = [line[:3] for line in csvreader] # prima era :2 -> non capisco se splitta in orizzontale o verticale
    return key_url_label_list  # Chop off header


def DownloadImage(key_url_label, out_dir):
    line = key_url_label
    #(key, url, label) = key_url_label
    key = line[0]
    url = line[1]
    label = line[2]
    out_lab = os.path.join(out_dir, label)
    filename = os.path.join(out_lab, '%s.jpg' % key)

    print(out_lab)
    if not os.path.isdir(out_lab):
      os.mkdir(out_lab)

    p = transforms.Compose([transforms.Resize((224, 224))])

    if os.path.exists(filename):
        print('Image %s already exists. Skipping download.' % filename)
        return

    try:
        response = urlopen(url)
        image_data = response.read()
    except:
        print('Warning: Could not download image %s from %s' % (key, url))
        return

    try:
        pil_image = Image.open(BytesIO(image_data))
    except:
        print('Warning: Failed to parse image %s' % key)
        return
    try:
        pil_image_rgb = pil_image.convert('RGB')
        pil_image_rgb = p(pil_image_rgb)
    except:
        print('Warning: Failed to convert image %s to RGB' % key)
        return

    try:
        pil_image_rgb.save(filename, format='JPEG')
    except:
        print('Warning: Failed to save image %s' % filename)
        return


def Run(data_file, out_dir):
    #if len(sys.argv) != 3:
    #    print('Syntax: %s <data_file.csv> <output_dir/>' % sys.argv[0])
    #    sys.exit(0)
    #(data_file, out_dir) = sys.argv[1:]

    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    key_url_label_list = ParseData(data_file)
    #pool = multiprocessing.Pool(processes=50)
    #pool.map(DownloadImage, key_url_label_list, out_dir)
    for element in key_url_label_list:
      #print(element)
      DownloadImage(element, out_dir)


#if __name__ == '__main__':
#    Run()


**Parameters**

In [ ]:
# change batch size for your GPU card !!!
glo_batch_size = 32

BATCH_SIZE = 32
NUM_EPOCHS = 30

**Data Data Preprocessing**

In [ ]:
#data_transform = transforms.Compose([
#    transforms.ToTensor(),
#    transforms.Normalize((0.1307,), (0.3081,)) # probably non dovremo normalizzare così
#])
# FORSE QUI CI SONO DELLA TRANSFORM SUPERFLUE
# LE SEGUENTI TRANSFORM SONO PRESE DAL CODICE DELLA VGG
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(224),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                  # Remember this when applying different transformations, otherwise you get an error
                                      transforms.Resize((28,28)),  # PROVIAMO CON LA DIMENSIONE DEL MNIST
                                      #transforms.Resize((224,224)),
                                      #transforms.Grayscale(num_output_channels=1),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
#                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Normalizes tensor with mean and standard deviation
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])
# saranno da calcolare i valori del normalize, li calcoliamo sul dataset? Sulle intero landmark o solo su quelle filtrate?

# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(224), # prima era scritto a 256
                                      transforms.CenterCrop(224),
                                      transforms.Resize((28,28)),  # PROVIAMO CON LA DIMENSIONE DEL MNIST
                                      #transforms.Resize((224,224)),
                                      #transforms.Grayscale(num_output_channels=1),
                                      transforms.ToTensor(),
#                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))                                   
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])




**Prepare Dataset**

In [ ]:
FMT = '%H:%M:%S'
now = datetime.now()
time1_before_download = now.strftime(FMT)
print(time1_before_download)

# same code as VGG16
# Clone github repository with data
if not os.path.isdir('./landmark-recognition'):
  !git clone https://github.com/davidetadz/landmark-recognition.git

data_file = './landmark-recognition/train_filter.csv'
out_dir = './landmark-recognition/train_img_folder/'

Run(data_file, out_dir)

DATA_DIR = 'landmark-recognition/train_img_folder/'

# Prepare Pytorch train/test Datasets
train_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=train_transform) # probably da cambiare e usare le funzioni che ho scritto
test_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=eval_transform)

train_indexes = [idx for idx in range(len(train_dataset)) if idx % 5]
test_indexes = [idx for idx in range(len(test_dataset)) if not idx % 5]

train_dataset = Subset(train_dataset, train_indexes)
test_dataset = Subset(test_dataset, test_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

now = datetime.now()
time2_after_download = now.strftime(FMT)
print(time2_after_download)


11:00:44
Cloning into 'landmark-recognition'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 28 (delta 0), reused 5 (delta 0), pack-reused 23
Unpacking objects: 100% (28/28), done.
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recognition/train_img_folder/456
./landmark-recogn

**Prepare Dataloaders**

In [ ]:
#train_loader = DataLoader(datasets.MNIST('mnist', train=True, download=True, transform=data_transform),
#                                           batch_size=glo_batch_size, shuffle=True
#                                           )
#test_loader = DataLoader(datasets.MNIST('mnist', train=False, download=True, transform=data_transform),
#                                          batch_size=glo_batch_size, shuffle=True
#                                          )


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True) #num_workers era 4, ma dava un errore coi thread, provo 0 
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

**CapsNet, Route and Caps code**

In [ ]:
# noi vogliamo molto più di 10 output, dobbiamo cambiare l'output della rete
class CapsNet(nn.Module):
    conv1_kernel_size = 9
    conv1_kernel_num = 256
    conv1_stride = 1

    caps1_conv_kernel_size = 9
    caps1_conv_kernel_num = 32
    caps1_conv1_stride = 2
    caps1_num = 9 # Era 8

    output_num = 50 # prima era 10 (immagino perche ci sono 10 numeri nel MNIST), provo a mettere 50
    output_size = 16

    batch_size = glo_batch_size

    def __init__(self, dataloader):
        super(CapsNet, self).__init__()
        self.dataloader = dataloader

        self.build()

    def create_cell_fn(self):
        """
        create sub-network inside a capsule.
        :return:
        """
        conv1 = nn.Conv2d(self.conv1_kernel_num, self.caps1_conv_kernel_num, kernel_size=self.caps1_conv_kernel_size,
                          stride=self.caps1_conv1_stride)
        # relu = nn.ReLU(inplace = True)
        # net = nn.Sequential(conv1, relu)
        return conv1

    def build(self):
        # Aggiungo altro strato convoluzionale
        #conv_pre = nn.Conv2d(3, self.conv1_kernel_num, kernel_size=self.conv1_kernel_size, stride=self.conv1_stride) 
        #conv_pre = nn.Conv2d(3, 3, kernel_size=self.conv1_kernel_size, stride=self.conv1_stride) 
        #relu_pre = nn.ReLU(inplace=True)

        conv1 = nn.Conv2d(3, self.conv1_kernel_num, kernel_size=self.conv1_kernel_size, stride=self.conv1_stride) # prima c'era 1 input channel
        relu1 = nn.ReLU(inplace=True)

        caps1_cells = [self.create_cell_fn() for i in range(self.caps1_num)]
        cap1 = Caps(caps1_cells)

        route1 = Route(self.caps1_num, 6 * 6 * self.caps1_conv_kernel_num, self.output_num, self.output_size,
                       batch_size=self.batch_size)

        self.net = nn.Sequential(conv1, relu1, cap1, route1) # Aggiungere conv_pre e relu_pre nel caso venissero usati

    def forward(self, input):
        return self.net(input)

    def margin_loss(self, input, target):
        v_mod = torch.sqrt(torch.mul(input, input).sum(dim=2, keepdim=True))

        m_plus = 0.9
        m_minus = 0.1
        zero_val = Variable(torch.zeros(1))
        max_l = torch.max(m_plus - v_mod, zero_val).view(self.batch_size, -1)
        max_r = torch.max(v_mod - m_minus, zero_val).view(self.batch_size, -1)
        Lc = target * max_l + 0.5 * (1 - target) * (max_r)
        Lc = Lc.sum(dim=1).mean()

        return Lc


class Route(nn.Module):

    def __init__(self, in_caps_num, in_caps_size, out_caps_num, out_caps_size, batch_size):
        super(Route, self).__init__()
        self.in_caps_num = in_caps_num
        self.in_caps_size = in_caps_size
        self.out_caps_num = out_caps_num
        self.out_caps_size = out_caps_size
        self.batch_size = glo_batch_size

        # we can not use batch_size for W parameters as the network does not include batch factor.
        self.W = nn.Parameter(torch.randn(1, in_caps_size, out_caps_num, out_caps_size, in_caps_num))

    def softmax(self, input, dim):
        input_ex = torch.exp(input)
        return input_ex / input_ex.sum(dim, keepdim=True)

    def squash(self, input):
        mod_sq = torch.sum(input ** 2, dim=2, keepdim=True)
        mod = torch.sqrt(mod_sq)
        return (mod_sq / (1 + mod_sq)) * (input / mod)

    def forward(self, input):
        # input (batch, in_caps_num, in_caps_size) => (batch, in_caps_size, in_caps_num)
        input = torch.transpose(input, 1, 2)
        # input (batch, vectorsin_caps_size, in_caps_num) =.(batch, in_caps_size, out_caps_num, in_caps_num, 1)
        input = torch.stack([input] * self.out_caps_num, dim=2).unsqueeze(4)
        # u_hat     : (batch, in_caps_size, out_caps_num, out_caps_size,     1)
        #          = (batch, in_caps_size, out_caps_num, out_caps_size, in_caps_num)
        #          * (batch, in_caps_size, out_caps_num, in_caps_num,       1)
        u_hat = torch.matmul(torch.cat([self.W] * self.batch_size, 0), input)
        # initialzie b_ij according to prior prob distribution
        # b_ij (1, in_caps_size, out_caps_num, 1), do not include batch_size
        b_ij = Variable(torch.zeros(1, self.in_caps_size, self.out_caps_num, 1))

        # start routing now.
        for _ in range(3): # questo 3 potrebbe riferirsi all'RGB
            # convert to coupling parameters, (batch_size, in_caps_size, out_caps_num, 1)
            c_ij = self.softmax(b_ij, dim=2)
            c_ij = torch.cat([c_ij] * self.batch_size, dim=0).unsqueeze(4)

            # Here using broadcasting mechnism.
            # s_j : (batch, in_caps_size, out_caps_num, out_caps_size, 1)
            #    = (batch, in_caps_size, out_caps_num, 1,             1)
            #    * (batch, in_caps_size, out_caps_num, out_caps_size, 1)
            # sum:  (batch,      1      , out_caps_num, out_caps_size, 1)
            s_j = torch.mul(c_ij, u_hat).sum(dim=1, keepdim=True)
            # squash s_j to v_j (batch, 1, out_caps_num, out_caps_size, 1)
            v_j = self.squash(s_j)
            # in order to satifiy u_hat * v_j, we expand its dimension
            # => (batch, in_caps_size, out_caps_num, out_caps_size, 1)
            v_j_ext = torch.cat([v_j] * self.in_caps_size, dim=1)
            # u_hat transpose => (batch, in_caps_size, out_caps_num, 1, out_caps_size)
            # v_j_ext            (batch, in_caps_size, out_caps_num, out_caps_size, 1)
            # matmul          => (batch, in_caps_size, out_caps_num, 1   , 1)
            # seueeze         => (batch, in_caps_size, out_caps_num, 1)
            # mean            => (  1  , in_caps_size, out_caps_num, 1)
            uv = torch.matmul(u_hat.transpose(3, 4), v_j_ext).squeeze(4).mean(dim=0, keepdim=True)
            # update b_ij
            b_ij = b_ij + uv

        # return (batch, out_caps_num, out_caps_size, 1)
        return v_j.squeeze(1)


class Caps(nn.Module):
    """
    Capsule layer, this layer is a wrapper of any kinds of sub-layer inside single capsule. When initialized, it received a create_cell_fn to create each 
    sub network for each capsules and compute each capsule output.
    In the feature, we can put all current network as a capsule unit.
    """

    def __init__(self, cells):
        super(Caps, self).__init__()
        self.cells = cells
        self.caps_num = len(cells)

        for i, cell in enumerate(cells):
            self.add_module("subnet" + str(i), cell)

    def forward(self, input):
        # u=[val] : val: (batch, channels, height, width)
        u = [self.cells[i](input) for i in range(self.caps_num)]
        # stack the caps_num axis before channels axis.
        # => (batch, caps_num, channels, height, width)
        u = torch.stack(u, dim=1)
        # flat to (batch, caps_num, output)
        u = u.view(input.size(0), self.caps_num, -1)
        # squash output
        return self.squash(u)

    def squash(self, input):
        mod_sq = torch.sum(input ** 2, dim=2, keepdim=True)
        mod = torch.sqrt(mod_sq)
        return (mod_sq / (1 + mod_sq)) * (input / mod)


def to_one_hot(x, length):
    batch_size = x.size(0)
    x_one_hot = torch.zeros(batch_size, length)
    for i in range(batch_size):
        x_one_hot[i, x[i]] = 1.0
    return x_one_hot



**Main**

In [ ]:
#if __name__ == '__main__':
net = CapsNet(train_loader)
print(net)

optimizer = optim.Adam(net.parameters(), lr=1e-2) #Prima il LR era 5e-4
tb = SummaryWriter()

best_accuracy = 0

if os.path.exists('caps.mdl'):
    net = torch.load('caps.mdl')
    print('loaded mdl yet.')

for epoch in range(NUM_EPOCHS):
    print('Starting epoch {}'.format(epoch+1))
    net.train()
    batch_idx = -1
    for (data, target) in tqdm(train_loader):
        batch_idx += 1
        target_onehot = to_one_hot(target, 50) # prima era 10, forse è il numero di output quindi metto 50
        data, target = Variable(data), Variable(target_onehot)

        optimizer.zero_grad()
        output = net(data)
        loss = net.margin_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0 and batch_idx != 0: # prima era modulo 50 (why?)
            tb.add_scalar('loss', loss.item())
            print('loss batch index {}: {}'.format(batch_idx, loss.item()))


    now = datetime.now()
    time3_after_training = now.strftime(FMT)
    print(time3_after_training)

    net.eval()
    correct_prediction = 0.0
    total_counter = 0

    batch_idx = -1
    for (data, target) in tqdm(test_loader):
        batch_idx += 1
        data, target = Variable(data), Variable(target.type(torch.LongTensor))

        # pred [batch, out_caps_num, out_caps_size, 1]
        pred = net(data)
        # pred_mod [batch, out_caps_num, 1, 1] => [batch, out_caps_num]
        pred_mod = pred.mul(pred).sum(dim=2).sqrt().squeeze(2)
        # v1 [batch]
        _, v1 = torch.max(pred_mod, dim=1)
        correct_prediction += target.eq(v1).sum().data.cpu().numpy()
        total_counter += glo_batch_size

        if batch_idx % 20 == 0: # prima era modulo 1000
            tb.add_scalar('accuracy', correct_prediction / (total_counter))
            break

    print(epoch+1, 'test accuracy:', correct_prediction / (total_counter))

    now = datetime.now()
    time4_after_testing = now.strftime(FMT)
    print(time4_after_testing)

    #if best_accuracy < correct_prediction / (total_counter):
    #    best_accuracy = correct_prediction / (total_counter)
    #    torch.save(net, 'caps.mdl')
    #    print('saved to caps.mdl file.')

    tb.close()



delta_download = datetime.strptime(time2_after_download, FMT) - datetime.strptime(time1_before_download, FMT)
delta_training = datetime.strptime(time3_after_training, FMT) - datetime.strptime(time2_after_download, FMT)
delta_testing = datetime.strptime(time4_after_testing, FMT) - datetime.strptime(time3_after_training, FMT)

print('Time to download:\t{}'.format(delta_download))
print('Time to train:\t\t{}'.format(delta_training))
print('Time to test:\t\t{}'.format(delta_testing))

  0%|          | 0/108 [00:00<?, ?it/s]

CapsNet(
  (net): Sequential(
    (0): Conv2d(3, 256, kernel_size=(9, 9), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Caps(
      (subnet0): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet1): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet2): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet3): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet4): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet5): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet6): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet7): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
      (subnet8): Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
    )
    (3): Route()
  )
)
Starting epoch 1


 19%|█▉        | 21/108 [01:16<05:04,  3.50s/it]

loss batch index 20: 0.8226540088653564


 38%|███▊      | 41/108 [02:25<03:52,  3.47s/it]

...

loss batch index 80: 0.1948087215423584


 94%|█████████▎| 101/108 [05:39<00:23,  3.35s/it]

loss batch index 100: 0.13447076082229614


  0%|          | 0/28 [00:00<?, ?it/s]

14:31:33
30 test accuracy: 0.71875
14:31:34
Time to download:	0:27:16
Time to train:		3:03:33
Time to test:		0:00:01
